# Generate QA Embedding Pairs 

## Create Prompt

In [25]:
prompt_it = """
Le informazioni sul contesto sono riportate di seguito.

---------------------
{context_str}
---------------------

Date le informazioni sul contesto e non le conoscenze pregresse.
generare solo domande basate sulla domanda seguente.

Siete un insegnante/professore. Il vostro compito è quello di impostare 
{num_questions_per_chunk} per un prossimo quiz.
Le domande devono essere di natura diversa nel documento. 
Limitare le domande alle informazioni di contesto fornite. 
Le domande devono essere in italiano."
"""

## Generate QA pairs wiht local model (e.g., Ollama)

### read files

In [26]:
from loguru import logger
from llama_index.core import SimpleDirectoryReader

directory = r"doc/"
required_exts = [".txt"]

documents = SimpleDirectoryReader(input_dir=directory, required_exts=required_exts).load_data()

logger.info(f"Number of documents loaded: {len(documents)}")

`llama-index-readers-file` package not found, some file readers will not be available if not provided by the `file_extractor` parameter.


2025-07-23 11:47:23.323 | INFO     | __main__:<module>:9 - Number of documents loaded: 1


### parse documents

In [27]:
from llama_index.core.node_parser import SentenceSplitter

class parserDocuments:
    chunk_size = 1000  # Number of characters per chunk
    train_size = 1.0  # Percentage of data to use for training

parser_params = parserDocuments()
parser = SentenceSplitter(chunk_size=parser_params.chunk_size)
nodes_train = parser.get_nodes_from_documents(documents= documents[ : int(len(documents) * parser_params.train_size)])
logger.info(f"Number of nodes created: {len(nodes_train)}")

2025-07-23 11:47:23.484 | INFO     | __main__:<module>:10 - Number of nodes created: 222


### generate QA pairs

In [28]:
from llama_index.llms.ollama import Ollama
from llama_index.finetuning import generate_qa_embedding_pairs

In [29]:
class configQA:
    filename = r"doc/IT_SWD_2025_212_1_IT.pdf"
    num_questions_per_chunk = 1
    output_path = "dataset.json"
    
qa_params= configQA()

In [30]:
class configOLLAMA:
    model_name = "llama3.1:latest"
    request_timeout = 240.0
    json_mode= True
    context_window = 4096  # Context window size for the LLM
  
ollama_params = configOLLAMA()

local_llm = Ollama(
                    model=ollama_params.model_name,
                    request_timeout=ollama_params.request_timeout,
                    #context_window=ollama_params.context_window,
                    json_mode=ollama_params.json_mode
                )

In [31]:
output_qa_filename_ = "QA/" + ollama_params.model_name + "_" + qa_params.output_path
output_qa_filename = output_qa_filename_.split(".json")[0].replace(".", "").replace(":", "_") + ".json"
output_qa_filename

'QA/llama31_latest_dataset.json'

In [32]:
generate_qa_embedding_pairs(
                            llm = local_llm,
                            nodes = nodes_train,
                            qa_generate_prompt_tmpl = prompt_it,
                            num_questions_per_chunk = qa_params.num_questions_per_chunk,
                            output_path = output_qa_filename
                            )

  0%|          | 0/222 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


  0%|          | 1/222 [01:07<4:08:44, 67.53s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


  1%|          | 2/222 [02:31<4:42:56, 77.17s/it]

...

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


  2%|▏         | 4/222 [05:58<5:39:27, 93.43s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


  2%|▏         | 5/222 [08:54<6:26:50, 106.96s/it]


KeyboardInterrupt: 